In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import gym
import matplotlib.pyplot as plt

In [2]:
class ReplayBuffer():
    def __init__(self, memSize, inputDims):
        self.memSize =  memSize
        self.memCntr = 0
        self.stateMem = np.zeros((self.memSize, *inputDims), dtype = np.float32)
        self.newStateMem = np.zeros((self.memSize, *inputDims), dtype = np.float32)
        self.actionMem = np.zeros(self.memSize, dtype = np.int32)
        self.rewardMem = np.zeros(self.memSize, dtype = np.float32)
        self.terminalMem = np.zeros(self.memSize, dtype = np.float32)

    def storeTransition(self, state, action, reward, state_, done):
        index = self.memCntr % self.memSize
        self.stateMem[index] = state
        self.newStateMem[index] = state_
        self.rewardMem[index] = reward
        self.actionMem[index] = action
        self.terminalMem[index] = 1 - int(done)
        self.memCntr += 1

    def sampleBuffer(self, batchSize):
        maxMem = min(self.memCntr, self.memSize)
        batch = np.random.choice(maxMem, batchSize, replace = False)
        states = self.stateMem[batch]
        states_ = self.newStateMem[batch]
        rewards = self.rewardMem[batch]
        actions = self.actionMem[batch]
        terminal = self.terminalMem[batch]

        return states, actions, rewards, states_, terminal



In [3]:
def buildDqn(lr, nActions, inputDims, fc1Dims, fc2Dims):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(fc1Dims, activation = 'relu'),
        tf.keras.layers.Dense(fc2Dims, activation = 'relu'),
        tf.keras.layers.Dense(nActions, activation = None)])
    model.compile(optimizer = Adam(learning_rate = lr), loss = 'mean_squared_error')

    return model

In [4]:
class Agent():
    def __init__(self, lr, gamma, nActions, epsilon, batchSize, 
                inputDims, epsilonDec = 1e-3, epsilonEnd = 0.01, 
                memSize = 1000000, fname = 'dqn_model.h5'):
        self.actionSpace = [i for i in range(nActions)]
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsMin = epsilonEnd
        self.batchSize = batchSize
        self.modelFile = fname
        self.epsilonDec = epsilonDec
        self.memory = ReplayBuffer(memSize, inputDims)
        self.qEval = buildDqn(lr, nActions, inputDims, 256, 256)

    def storeTransition(self, state, action, reward, newState, done):
        self.memory.storeTransition(state, action, reward, newState, done)

    def chooseAction(self, observation):
        if np.random.random() < self.epsilon:
            action = np.random.choice(self.actionSpace)
        else:
            state = np.array([observation])
            actions =self.qEval.predict(state)
            action = np.argmax(actions)
        
        return action

    def learn(self):
        if self.memory.memCntr < self.batchSize:
            return
        
        states, actions, rewards, states_, dones = self.memory.sampleBuffer(self.batchSize)

        qEval = self.qEval.predict(states)
        qNext = self.qEval.predict(states_)

        qTarget = np.copy(qEval)
        batchIndex = np.arange(self.batchSize, dtype = np.int32)

        qTarget[batchIndex, actions] = rewards + self.gamma * np.max(qNext, axis = 1) * dones

        self.qEval.train_on_batch(states, qTarget)

        self.epsilon = self.epsilon - self.epsilonDec if self.epsilon > self.epsMin else self.epsMin

    def saveModel(self):
        self.qEval.save(self.modelFile)

    def loadModel(self):
        self.qEval = load_model(self.modelFile)


In [8]:
env = gym.make('CartPole-v1')
lr = 0.001
nGames = 500
agent = Agent(gamma = 0.99, epsilon = 1.0, lr = lr, inputDims = env.observation_space.shape, nActions = env.action_space.n,
              memSize = 1000000, batchSize = 64, epsilonEnd = 0.01)
scores = []
epsHist = []
for i in range(nGames):
    done = False
    score = 0
    observation = env.reset()
    while not done:
        action = agent.chooseAction(observation)
        observation_, reward, done, info = env.step(action)
        score += reward
        agent.storeTransition(observation, action, reward, observation_, done)
        observation = observation_
        agent.learn()
    epsHist.append(agent.epsilon)
    scores.append(score)

    avgScore = np.mean(scores[-100:])
    print('episode: ', i, 'score %.2f' % score, 'average score %.2f' % avgScore, 'epsilon %.2f' % agent.epsilon)

    filename = 'pole'

episode:  0 score 12.00 average score 12.00 epsilon 1.00
episode:  1 score 21.00 average score 16.50 epsilon 1.00
episode:  2 score 12.00 average score 15.00 epsilon 1.00
episode:  3 score 13.00 average score 14.50 epsilon 1.00
episode:  4 score 9.00 average score 13.40 epsilon 1.00
episode:  5 score 25.00 average score 15.33 epsilon 0.97
episode:  6 score 12.00 average score 14.86 epsilon 0.96
episode:  7 score 27.00 average score 16.38 epsilon 0.93
episode:  8 score 20.00 average score 16.78 epsilon 0.91
episode:  9 score 19.00 average score 17.00 epsilon 0.89
episode:  10 score 14.00 average score 16.73 epsilon 0.88
episode:  11 score 25.00 average score 17.42 epsilon 0.85
episode:  12 score 13.00 average score 17.08 epsilon 0.84
episode:  13 score 15.00 average score 16.93 epsilon 0.83
episode:  14 score 44.00 average score 18.73 epsilon 0.78
episode:  15 score 16.00 average score 18.56 epsilon 0.77
episode:  16 score 24.00 average score 18.88 epsilon 0.74
episode:  17 score 10.00 

In [6]:
agent.saveModel()

In [9]:
plt.plot(scores, 'go-')
plt.rcParams["figure.figsize"] = (25,5)
plt.show()